# Json import to openSearch

In [ ]:
import createRecipesMap as crm

crm.CreateRecipesMap("../jsonData/recipe_queries_results.json", '../jsonData/recipesMap.json').createMap()

# Login to openSearch

In [ ]:
import loginOpenSearch as lop

login = lop.LoginOpenSearch("../jsonData/openSearchConfig.json").login()
    
client = login[0]
index_name = login[1]

# Create index structure

In [ ]:
import createIndexes as ci

ci.CreateIndexes(client, index_name).createIndexStructure()

# Choose the tokenizer and model that will be used in the embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")
model = AutoModel.from_pretrained("sentence-transformers/msmarco-distilbert-base-v2")

In [ ]:
import computeEmbeddings as ce

ce.ComputeEmbeddings("../jsonData/recipesMap.json","../jsonData/recipesEmbeddingsMap.json", tokenizer, model).createMap()

In [ ]:
import indexRecipes as ir

ir.IndexRecipes("../jsonData/recipesMap.json", "../jsonData/recipesEmbeddingsMap.json", client, index_name).indexRecipes()


In [ ]:
import search as s

s.Search(client, index_name, tokenizer, model).queryOpenSearch('Holiday Salad', 5,None, None, ["salads"], ["lupine"], None)

In [ ]:
adfasdfdafdsafd
if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = "600s"
    )
    print('\nDeleting index:')
    print(response)